# Dust Evolution

Dust evolution in `DustPy` consists of three parts: coagulation, hydrodynamics, and external sources.  
Their source terms are stored in `Simulation.dust.S`.

In [1]:
from dustpy import Simulation
sim = Simulation()
sim.initialize()

In [2]:
sim.dust.S

Group (Sources)
---------------
    coag         : Field (Coagulation sources [g/cm²/s])
    ext          : Field (External sources [g/cm²/s])
    hyd          : Field (Hydrodynamic sources [g/cm²/s])
    tot          : Field (Tot sources [g/cm²/s])
  -----

The total source terms `Simulation.dust.S.tot` is used as source terms for the integration method.

The dust is by default integrated with the 5th-order adaptive Cash-Karp method provided by `Simframe`. Since the gas is by default integrated with an implicit method it is unconditionally stable and uses the time step set by the dust evolution.

In [3]:
sim.integrator.instructions

[Instruction (Dust: explicit 5th-order adaptive Cash-Karp method),
 Instruction (Gas: implicit 1st-order Euler method)]

## Coagulation

Please see the previous chapter for details on coagulation.  
The coagulation source terms are stored in `Simulation.dust.S.coag`.

### Turning off Fragmentation

If you want to turn off fragmentation (but not sticking), set the sticking probabilities to $1$, the fragmentation probabilities to $0$, and unset the updater of the probabilities.

In [4]:
sim.dust.p.stick = 1.
sim.dust.p.frag = 0.
sim.dust.p.updater = None

**Note:** You could also set the fragmentation velocities to a very large value that will never be reached. However, this will still calculate the probabilies at every timestep, even though they are constant in this case and consume valuable computational resources.

### Turning off Coagulation

If you want to turn off coagulation in total, set the coagulation sources to $0$ and unset the updater.

In [5]:
sim.dust.S.coag = 0.
sim.dust.S.coag.updater = None

You can furthermore unset the updaters of the other fields, that are only used for coagulation to save computational resources.

In [6]:
sim.dust.p.updater = None
sim.dust.v.rel.updater = None
sim.dust.v.frag.updater = None
sim.dust.kernel.updater = None

## Hydrodynamics

`DustPy` solves the following equation for dust transport

$\frac{\partial}{\partial t} \Sigma_\mathrm{d} + \frac{1}{r} \frac{\partial}{\partial r} \left( r\Sigma_\mathrm{d}v_\mathrm{rad} \right) - \frac{1}{r} \frac{\partial}{\partial r} \left[ D \Sigma_\mathrm{g} \frac{\partial}{\partial r} \left( \frac{\Sigma_\mathrm{d}}{\Sigma_\mathrm{g}} \right) \right] = 0$

The dust hydrodynamics consists of an advective and an diffusive term. The algorithm calculates the advective and diffusive fluxes separately at the grid cell interfaces. Furthermore, the sum of both is calculated.

In [7]:
sim.dust.Fi

Group (Fluxes)
--------------
    adv          : Field (Advective flux [g/cm/s)
    diff         : Field (Diffusive flux [g/cm/s)
    tot          : Field (Total flux [g/cm/s)
  -----

### Advection

The advective fluxes at the grid interfaces are calculated as

$F_\mathrm{adv} = v_\mathrm{rad} \cdot \Sigma_\mathrm{d}$

and interpolated onto the grid cell interfaces. The default boundary condition assumes constant velocity.

### Turning off Advection

To turn off advection, you can set the advective fluxes to $0$ and unset the updater.

In [8]:
sim.dust.Fi.adv = 0
sim.dust.Fi.adv.updater = None

However, this will still calculate the radial velocities, which will have an effect on the collision velocities caused by radial drift. If you do not want radial drift to contribute to the relative velocities, you also have to unset it.

In [9]:
sim.dust.v.rad = 0
sim.dust.v.rad.updater = None

You might also want to unset the relative velocities caused by azimuthal drift.  
If you have set a backreaction mechanism you should think if you need to unset it for your model, too.

### Diffusion

The diffusive fluxes are given by

$F_\mathrm{diff} = -D\Sigma_\mathrm{g}\nabla\frac{\Sigma_\mathrm{d}}{\Sigma_\mathrm{gas}}$

A flux limiter limits the maximum diffusive flux to the diffusive RMS velocity of that dust species. The diffusive flux through the boundaries is set to zero.

### Turning off Diffusion

To turn off dust diffusion you can set the diffusive fluxes to $0$ and unset the updater.

In [10]:
sim.dust.Fi.diff = 0.
sim.dust.Fi.diff.updater = None

You can furthermore unset the updater of the diffusivity, since it's not needed.

In [11]:
sim.dust.D.updater = None

### Turning off Hydrodynamics

If you want to turn off dust hydrodynamics in total, you can simply set hydrodynamic source terms of the dust to $0$ and unset the updater.

In [12]:
sim.dust.S.hyd = 0.
sim.dust.S.hyd.updater = None

You can also unset the updater of the fluxes.

In [13]:
sim.dust.Fi.updater = None

Depending on the aim of your simulation, you can unset the updaters of other fields, if you don't need them.

## External Sources

`DustPy` has the possibility to add external source/loss terms that you can set. With those you can for example add dust in case of infall or remove dust in case of planetesimal formation or pebble accretion.

By default the external sources are $0$ without a set updater.

## Turning off Dust Evolution

If you want to deactivate dust evolution as a whole you can set the total source terms to $0$ and unset the updater. This will leave `Simulation.dust.Sigma` unchanged throughout your simulation.

In [14]:
sim.dust.S.tot = 0.
sim.dust.S.updater = None

You can also deactivate other fields, that are not needed in that case, as described above.

Another possibility of turning off dust evolution is by removing the integration instruction from the instruction set.

In [15]:
sim.integrator.instructions

[Instruction (Dust: explicit 5th-order adaptive Cash-Karp method),
 Instruction (Gas: implicit 1st-order Euler method)]

In [16]:
del(sim.integrator.instructions[0])

In [17]:
sim.integrator.instructions

[Instruction (Gas: implicit 1st-order Euler method)]

Keep in mind that the updaters of all fields within `Simulation.dust` will still be called and the fields will be updated unless their updaters are unset. This will on one hand cost computational ressources, and on the other hand could influence the physics in an unwanted way.

For example the dust backreaction coefficients will still be calculated and can have an effect on the gas, even though the dust is in principle not moving.

Always keep in mind what you want to achieve by turning off dust evolution and unset the updaters of certain fields, if required.

**Note:** If you remove the dust integration instruction, you have to provide a function that calculates the timestep, since by default `DustPy` uses the timestep the adaptive dust integration scheme suggests. See the chapter about gas evolution tests for details.